In [ ]:
#@markdown **Step 1:** Check which GPU you've been allocated.

#@markdown You want a P100, V100 or T4. 
#@markdown If you get a P4 or K80, factory reset the runtime and try again.
!nvidia-smi -L
!nvidia-smi

GPU 0: Tesla T4 (UUID: GPU-fb5c47d7-eed3-7389-bae5-5f5d26cf5097)
Sun Aug 14 10:08:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                       

In [ ]:
#@markdown **Step 2:** Mount Google Drive.
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
#@markdown **Step 3:** Configure training data paths. Upload the following to your Drive and change the paths below:
#@markdown * A dataset of .wav files, packaged as a .zip or .tar file
#@markdown * Training and validation filelists, in LJSpeech format with relative paths (note: ARPABET transcripts are not supported)
#@markdown * An output path for checkpoints

import os

dataset = "/content/drive/My Drive/talknet/wavs.zip" #@param {type:"string"}
train_filelist = "/content/drive/My Drive/talknet/training_list.txt" #@param {type:"string"}
val_filelist = "/content/drive/My Drive/talknet/val_list.txt" #@param {type:"string"}
output_dir = "/content/drive/My Drive/talknet/trained/mizuki" #@param {type:"string"}
assert os.path.exists(dataset), "Cannot find dataset"
assert os.path.exists(train_filelist), "Cannot find training filelist"
assert os.path.exists(val_filelist), "Cannot find validation filelist"
if not os.path.exists(output_dir):
   os.makedirs(output_dir)
print("OK")


OK


In [ ]:
#@markdown **Step 4:** Download NVIDIA NeMo.
!pip install gdown --upgrade
%tensorflow_version 2.x
import os
import time
import gdown

os.chdir('/content')
!apt-get install sox libsndfile1 ffmpeg
!pip install wget unidecode tensorflow==2.4.1 tensorboardX pysptk frozendict torch_stft torchvision==0.9.1 torchaudio==0.8.1 torchtext==0.9.1 pytorch-lightning==1.3.8 kaldiio pydub pyannote.audio g2p_en pesq pystoi crepe ffmpeg-python
!pip install torchmetrics==0.6.0
!python -m pip install git+https://github.com/SortAnon/NeMo.git
!git clone -q https://github.com/SortAnon/hifi-gan.git

!mkdir -p conf && cd conf \
&& wget https://raw.githubusercontent.com/SortAnon/NeMo/main/examples/tts/conf/talknet-durs.yaml \
&& wget https://raw.githubusercontent.com/SortAnon/NeMo/main/examples/tts/conf/talknet-pitch.yaml \
&& wget https://raw.githubusercontent.com/SortAnon/NeMo/main/examples/tts/conf/talknet-spect.yaml \
&& cd ..

# Download pre-trained models
zip_path = "tts_en_talknet_1.0.0rc1.zip"
for i in range(10):
    if not os.path.exists(zip_path) or os.stat(zip_path).st_size < 100:
        gdown.download(
            "https://drive.google.com/uc?id=19wSym9mNEnmzLS9XdPlfNAW9_u-mP1hR",
            zip_path,
            quiet=False,
        )
!unzip -qo {zip_path}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.5.1-py3-none-any.whl size=14951 sha256=0343aa1386558d7c78b539e033cc5ebc31b44c77b851898c33e7495742e94714
  Stored in directory: /root/.cache/pip/wheels/3d/ec/b0/a96d1d126183f98570a785e6bf8789fca559853a9260e928e1
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed a

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 329 kB 5.0 MB/s 
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 0.9.3
    Uninstalling torchmetrics-0.9.3:
      Successfully uninstalled torchmetrics-0.9.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SortAnon/NeMo.git to /tmp/pip-req-build-00akeh84
  Running command git clone -q https://github.com/SortAnon/NeMo.git /tmp/pip-req-build-00akeh84
     |████████████████████████████████| 13.1 MB 5.1 MB/s 
     |████████████████████████████████| 109 kB 71.2 MB/s 
  Using cached omegaconf-2.2.2-py3-none-any.whl (79 kB)
     |████████████████████████████████| 151 kB 74.8 MB/s 
     |████████████████████████████████| 4.7 MB 46.2 MB/s 
     |████████████████████████████████| 1.3 MB 56.8 MB/s 
     |████████████████████████████████| 769 kB 76

Downloading...
From: https://drive.google.com/uc?id=19wSym9mNEnmzLS9XdPlfNAW9_u-mP1hR
To: /content/tts_en_talknet_1.0.0rc1.zip
100%|██████████| 51.2M/51.2M [00:00<00:00, 66.6MB/s]


In [ ]:
#@markdown **Step 5:** Dataset processing, part 1.

#@markdown If this step fails, try the following:
#@markdown * Make sure your filelists are correct. They should have relative 
#@markdown paths that match the contents of the archive.

import os
import shutil
import sys
import json
import nemo
import torch
import torchaudio
import numpy as np
from pysptk import sptk
from pathlib import Path
from tqdm.notebook import tqdm
import ffmpeg

def fix_transcripts(inpath):
    found_arpabet = False
    found_grapheme = False
    with open(inpath, "r", encoding="utf8") as f:
        lines = f.readlines()
    with open(inpath, "w", encoding="utf8") as f:
        for l in lines:
            if l.strip() == "":
                continue
            if "{" in l:
                if not found_arpabet:
                    print("Warning: Skipping ARPABET lines (not supported).")
                    found_arpabet = True
            else:
                f.write(l)
                found_grapheme = True
    assert found_grapheme, "No non-ARPABET lines found in " + inpath

def generate_json(inpath, outpath):
    output = ""
    sample_rate = 22050
    with open(inpath, "r", encoding="utf8") as f:
        for l in f.readlines():
            lpath = l.split("|")[0].strip()
            if lpath[:5] != "wavs/":
                lpath = "wavs/" + lpath
            size = os.stat(
                os.path.join(os.path.dirname(inpath), lpath)
            ).st_size
            x = {
                "audio_filepath": lpath,
                "duration": size / (sample_rate * 2),
                "text": l.split("|")[1].strip(),
            }
            output += json.dumps(x) + "\n"
        with open(outpath, "w", encoding="utf8") as w:
            w.write(output)

def convert_to_22k(inpath):
    if inpath.strip()[-4:].lower() != ".wav":
        print("Warning: " + inpath.strip() + " is not a .wav file!")
        return
    ffmpeg.input(inpath).output(
        inpath + "_22k.wav",
        ar="22050",
        ac="1",
        acodec="pcm_s16le",
        map_metadata="-1",
        fflags="+bitexact",
    ).overwrite_output().run(quiet=True)
    os.remove(inpath)
    os.rename(inpath + "_22k.wav", inpath)

# Extract dataset
os.chdir('/content')
if os.path.exists("/content/wavs"):
    shutil.rmtree("/content/wavs")
os.mkdir("wavs")
os.chdir("wavs")
if dataset[-4:] == ".zip":
    !unzip -q "{dataset}"
elif dataset[-4:] == ".tar":
    !tar -xf "{dataset}"
else:
    raise Exception("Unknown extension for dataset")
if os.path.exists("/content/wavs/wavs"):
    shutil.move("/content/wavs/wavs", "/content/tempwavs")
    shutil.rmtree("/content/wavs")
    shutil.move("/content/tempwavs", "/content/wavs")

# Filelist for preprocessing
os.chdir('/content')
shutil.copy(train_filelist, "trainfiles.txt")
shutil.copy(val_filelist, "valfiles.txt")
fix_transcripts("trainfiles.txt")
fix_transcripts("valfiles.txt")
seen_files = []
with open("trainfiles.txt") as f:
    t = f.read().split("\n")
with open("valfiles.txt") as f:
    v = f.read().split("\n")
    all_filelist = t[:] + v[:]
with open("/content/allfiles.txt", "w") as f:
    for x in all_filelist:
        if x.strip() == "":
            continue
        if x.split("|")[0] not in seen_files:
            seen_files.append(x.split("|")[0])
            f.write(x.strip() + "\n")

# Ensure audio is 22k
print("Converting audio...")
for r, _, f in os.walk("/content/wavs"):
    for name in tqdm(f):
        convert_to_22k(os.path.join(r, name))

# Convert to JSON
generate_json("trainfiles.txt", "trainfiles.json")
generate_json("valfiles.txt", "valfiles.json")
generate_json("allfiles.txt", "allfiles.json")

print("OK")

Converting audio...


  0%|          | 0/464 [00:00<?, ?it/s]

OK


In [ ]:
#@markdown **Step 6:** Dataset processing, part 2. This takes a while, but
#@markdown you only have to run this once per dataset (results are saved to Drive).

#@markdown If this step fails, try the following:
#@markdown * Make sure your dataset only contains WAV files.

# Extract phoneme duration

import json
from nemo.collections.asr.models import EncDecCTCModel
asr_model = EncDecCTCModel.from_pretrained(model_name="asr_talknet_aligner").cpu().eval()

def forward_extractor(tokens, log_probs, blank):
    """Computes states f and p."""
    n, m = len(tokens), log_probs.shape[0]
    # `f[s, t]` -- max sum of log probs for `s` first codes
    # with `t` first timesteps with ending in `tokens[s]`.
    f = np.empty((n + 1, m + 1), dtype=float)
    f.fill(-(10 ** 9))
    p = np.empty((n + 1, m + 1), dtype=int)
    f[0, 0] = 0.0  # Start
    for s in range(1, n + 1):
        c = tokens[s - 1]
        for t in range((s + 1) // 2, m + 1):
            f[s, t] = log_probs[t - 1, c]
            # Option #1: prev char is equal to current one.
            if s == 1 or c == blank or c == tokens[s - 3]:
                options = f[s : (s - 2 if s > 1 else None) : -1, t - 1]
            else:  # Is not equal to current one.
                options = f[s : (s - 3 if s > 2 else None) : -1, t - 1]
            f[s, t] += np.max(options)
            p[s, t] = np.argmax(options)
    return f, p


def backward_extractor(f, p):
    """Computes durs from f and p."""
    n, m = f.shape
    n -= 1
    m -= 1
    durs = np.zeros(n, dtype=int)
    if f[-1, -1] >= f[-2, -1]:
        s, t = n, m
    else:
        s, t = n - 1, m
    while s > 0:
        durs[s - 1] += 1
        s -= p[s, t]
        t -= 1
    assert durs.shape[0] == n
    assert np.sum(durs) == m
    assert np.all(durs[1::2] > 0)
    return durs

def preprocess_tokens(tokens, blank):
    new_tokens = [blank]
    for c in tokens:
        new_tokens.extend([c, blank])
    tokens = new_tokens
    return tokens

data_config = {
    'manifest_filepath': "allfiles.json",
    'sample_rate': 22050,
    'labels': asr_model.decoder.vocabulary,
    'batch_size': 1,
}

parser = nemo.collections.asr.data.audio_to_text.AudioToCharWithDursF0Dataset.make_vocab(
    notation='phonemes', punct=True, spaces=True, stresses=False, add_blank_at="last"
)

dataset = nemo.collections.asr.data.audio_to_text._AudioTextDataset(
    manifest_filepath=data_config['manifest_filepath'], sample_rate=data_config['sample_rate'], parser=parser,
)

dl = torch.utils.data.DataLoader(
    dataset=dataset, batch_size=data_config['batch_size'], collate_fn=dataset.collate_fn, shuffle=False,
)

blank_id = asr_model.decoder.num_classes_with_blank - 1

if os.path.exists(os.path.join(output_dir, "durations.pt")):
    print("durations.pt already exists; skipping")
else:
    dur_data = {}
    for sample_idx, test_sample in tqdm(enumerate(dl), total=len(dl)):
        log_probs, _, greedy_predictions = asr_model(
            input_signal=test_sample[0], input_signal_length=test_sample[1]
        )

        log_probs = log_probs[0].cpu().detach().numpy()
        seq_ids = test_sample[2][0].cpu().detach().numpy()

        target_tokens = preprocess_tokens(seq_ids, blank_id)

        f, p = forward_extractor(target_tokens, log_probs, blank_id)
        durs = backward_extractor(f, p)

        dur_key = Path(dl.dataset.collection[sample_idx].audio_file).stem
        dur_data[dur_key] = {
            'blanks': torch.tensor(durs[::2], dtype=torch.long).cpu().detach(), 
            'tokens': torch.tensor(durs[1::2], dtype=torch.long).cpu().detach()
        }

        del test_sample

    torch.save(dur_data, os.path.join(output_dir, "durations.pt"))

#Extract F0 (pitch)
import crepe
from scipy.io import wavfile

def crepe_f0(audio_file, hop_length=256):
    sr, audio = wavfile.read(audio_file)
    audio_x = np.arange(0, len(audio)) / 22050.0
    time, frequency, confidence, activation = crepe.predict(audio, sr, viterbi=True)

    x = np.arange(0, len(audio), hop_length) / 22050.0
    freq_interp = np.interp(x, time, frequency)
    conf_interp = np.interp(x, time, confidence)
    audio_interp = np.interp(x, audio_x, np.absolute(audio)) / 32768.0
    weights = [0.5, 0.25, 0.25]
    audio_smooth = np.convolve(audio_interp, np.array(weights)[::-1], "same")

    conf_threshold = 0.25
    audio_threshold = 0.0005
    for i in range(len(freq_interp)):
        if conf_interp[i] < conf_threshold:
            freq_interp[i] = 0.0
        if audio_smooth[i] < audio_threshold:
            freq_interp[i] = 0.0

    # Hack to make f0 and mel lengths equal
    if len(audio) % hop_length == 0:
        freq_interp = np.pad(freq_interp, pad_width=[0, 1])
    return torch.from_numpy(freq_interp.astype(np.float32))

if os.path.exists(os.path.join(output_dir, "f0s.pt")):
    print("f0s.pt already exists; skipping")
else:
    f0_data = {}
    with open("allfiles.json") as f:
        for i, l in enumerate(f.readlines()):
            print(str(i))
            audio_path = json.loads(l)["audio_filepath"]
            f0_data[Path(audio_path).stem] = crepe_f0(audio_path)

    # calculate f0 stats (mean & std) only for train set
    with open("trainfiles.json") as f:
        train_ids = {Path(json.loads(l)["audio_filepath"]).stem for l in f}
    all_f0 = torch.cat([f0[f0 >= 1e-5] for f0_id, f0 in f0_data.items() if f0_id in train_ids])

    F0_MEAN, F0_STD = all_f0.mean().item(), all_f0.std().item()        
    print("F0_MEAN: " + str(F0_MEAN) + ", F0_STD: " + str(F0_STD))
    torch.save(f0_data, os.path.join(output_dir, "f0s.pt"))
    with open(os.path.join(output_dir, "f0_info.json"), "w") as f:
        f.write(json.dumps({"FO_MEAN": F0_MEAN, "F0_STD": F0_STD}))

[NeMo W 2022-08-14 10:17:27 optimizers:47] Apex was not found. Using the lamb optimizer will error out.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
[NeMo W 2022-08-14 10:17:32 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text_dali.AudioToCharDALIDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.


[NeMo I 2022-08-14 10:17:32 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/asr_talknet_aligner/versions/1.0.0rc1/files/qn5x5_libri_tts_phonemes.nemo to /root/.cache/torch/NeMo/NeMo_1.0.2/qn5x5_libri_tts_phonemes/656c7439dd3a0d614978529371be498b/qn5x5_libri_tts_phonemes.nemo
[NeMo I 2022-08-14 10:17:35 common:676] Instantiating model from pre-trained checkpoint


[NeMo W 2022-08-14 10:17:35 features:230] Using torch_stft is deprecated and will be removed in 1.1.0. Please set stft_conv and stft_exact_pad to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-08-14 10:17:35 features:252] PADDING: 1
[NeMo I 2022-08-14 10:17:35 features:262] STFT using conv
[NeMo I 2022-08-14 10:17:43 modelPT:439] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.2/qn5x5_libri_tts_phonemes/656c7439dd3a0d614978529371be498b/qn5x5_libri_tts_phonemes.nemo.
[NeMo I 2022-08-14 10:17:44 collections:173] Dataset loaded with 440 files totalling 0.24 hours
[NeMo I 2022-08-14 10:17:44 collections:174] 0 files were filtered totalling 0.00 hours


  0%|          | 0/440 [00:00<?, ?it/s]

...

433
6/6 [==============================] - 0s 53ms/step
434
8/8 [==============================] - 0s 48ms/step


[NeMo W 2022-08-14 10:23:18 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/crepe/core.py:209: RuntimeWarning: invalid value encountered in true_divide
      frames /= np.std(frames, axis=1)[:, np.newaxis]
    


435
11/11 [==============================] - 0s 47ms/step
436
4/4 [==============================] - 0s 50ms/step
437
4/4 [==============================] - 0s 53ms/step
438


[NeMo W 2022-08-14 10:23:19 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/crepe/core.py:209: RuntimeWarning: invalid value encountered in true_divide
      frames /= np.std(frames, axis=1)[:, np.newaxis]
    


10/10 [==============================] - 0s 47ms/step
439
3/3 [==============================] - 0s 52ms/step
F0_MEAN: 253.9666748046875, F0_STD: 74.89042663574219


In [ ]:
#@markdown **Step 7:** Train duration predictor.

#@markdown If CUDA runs out of memory, try the following:
#@markdown * Click on Runtime -> Restart runtime, re-run step 3, and try again.
#@markdown * If that doesn't help, reduce the batch size (default 64).
batch_size = 32 #@param {type:"integer"}

epochs = 20
learning_rate = 1e-3
min_learning_rate = 3e-6
load_checkpoints = True

import os
from hydra.experimental import compose, initialize
from hydra.core.global_hydra import GlobalHydra
from omegaconf import OmegaConf
import pytorch_lightning as pl
from nemo.collections.common.callbacks import LogEpochTimeCallback
from nemo.collections.tts.models import TalkNetDursModel
from nemo.core.config import hydra_runner
from nemo.utils.exp_manager import exp_manager

def train(cfg):
    cfg.sample_rate = 22050
    cfg.train_dataset = "trainfiles.json"
    cfg.validation_datasets = "valfiles.json"
    cfg.durs_file = os.path.join(output_dir, "durations.pt")
    cfg.f0_file = os.path.join(output_dir, "f0s.pt")
    cfg.trainer.accelerator = "dp"
    cfg.trainer.max_epochs = epochs
    cfg.trainer.check_val_every_n_epoch = 5
    cfg.model.train_ds.dataloader_params.batch_size = batch_size
    cfg.model.validation_ds.dataloader_params.batch_size = batch_size
    cfg.model.optim.lr = learning_rate
    cfg.model.optim.sched.min_lr = min_learning_rate
    cfg.exp_manager.exp_dir = output_dir

    # Find checkpoints
    ckpt_path = ""
    if load_checkpoints:
      path0 = os.path.join(output_dir, "TalkNetDurs")
      if os.path.exists(path0):
          path1 = sorted(os.listdir(path0))
          for i in range(len(path1)):
              path2 = os.path.join(path0, path1[-(1+i)], "checkpoints")
              if os.path.exists(path2):
                  match = [x for x in os.listdir(path2) if "last.ckpt" in x]
                  if len(match) > 0:
                      ckpt_path = os.path.join(path2, match[0])
                      print("Resuming training from " + match[0])
                      break
    
    if ckpt_path != "":
        trainer = pl.Trainer(**cfg.trainer, resume_from_checkpoint = ckpt_path)
        model = TalkNetDursModel(cfg=cfg.model, trainer=trainer)
    else:
        warmstart_path = "/content/talknet_durs.nemo"
        trainer = pl.Trainer(**cfg.trainer)
        model = TalkNetDursModel.restore_from(warmstart_path, override_config_path=cfg)
        model.set_trainer(trainer)
        model.setup_training_data(cfg.model.train_ds)
        model.setup_validation_data(cfg.model.validation_ds)
        model.setup_optimization(cfg.model.optim)
        print("Warm-starting from " + warmstart_path)
    exp_manager(trainer, cfg.get('exp_manager', None))
    trainer.callbacks.extend([pl.callbacks.LearningRateMonitor(), LogEpochTimeCallback()])  # noqa
    trainer.fit(model)

GlobalHydra().clear()
initialize(config_path="conf")
cfg = compose(config_name="talknet-durs")
train(cfg)


[NeMo W 2022-08-14 10:24:58 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:43: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
      deprecation_warning(message=message)
    
[NeMo W 2022-08-14 10:24:58 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:48: UserWarning: 
    The version_base parameter is not specified.
    Please specify a compatability version level, or None.
    Will assume defaults for version 1.1
      caller_stack_depth=caller_stack_depth + 1,
    
[NeMo W 2022-08-14 10:24:58 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/hydra/experimental/compose.py:25: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
      deprecation_warning(message=message)
    
INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: Fal

[NeMo I 2022-08-14 10:24:59 modelPT:439] Model TalkNetDursModel was successfully restored from /content/talknet_durs.nemo.
[NeMo I 2022-08-14 10:24:59 collections:173] Dataset loaded with 395 files totalling 0.22 hours
[NeMo I 2022-08-14 10:24:59 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2022-08-14 10:24:59 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2022-08-14 10:24:59 collections:173] Dataset loaded with 45 files totalling 0.02 hours
[NeMo I 2022-08-14 10:24:59 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2022-08-14 10:24:59 modelPT:661] The lightning trainer received accelerator: dp. We recommend to use 'ddp' instead.


...

INFO:pytorch_lightning.utilities.distributed:Epoch 9, global step 129: val_acc reached 75.67676 (best 75.67676), saving model to "/content/drive/My Drive/talknet/trained/mizuki/TalkNetDurs/2022-08-14_10-24-59/checkpoints/TalkNetDurs--val_acc=75.68-epoch=9.ckpt" as top 3


Validating: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.distributed:Epoch 14, global step 194: val_acc reached 76.37921 (best 76.37921), saving model to "/content/drive/My Drive/talknet/trained/mizuki/TalkNetDurs/2022-08-14_10-24-59/checkpoints/TalkNetDurs--val_acc=76.38-epoch=14.ckpt" as top 3


Validating: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.distributed:Epoch 19, global step 259: val_acc reached 75.99054 (best 76.37921), saving model to "/content/drive/My Drive/talknet/trained/mizuki/TalkNetDurs/2022-08-14_10-24-59/checkpoints/TalkNetDurs--val_acc=75.99-epoch=19.ckpt" as top 3
INFO:pytorch_lightning.utilities.distributed:Saving latest checkpoint...


In [ ]:
#@markdown **Step 8:** Train pitch predictor.

#@markdown If CUDA runs out of memory, try the following:
#@markdown * Click on Runtime -> Restart runtime, re-run step 3, and try again.
#@markdown * If that doesn't help, reduce the batch size (default 64).
batch_size = 16 #@param {type:"integer"}
epochs = 50

import json

with open(os.path.join(output_dir, "f0_info.json"), "r") as f:
    f0_info = json.load(f)
    f0_mean = f0_info["FO_MEAN"]
    f0_std = f0_info["F0_STD"]

learning_rate = 1e-3
min_learning_rate = 3e-6
load_checkpoints = True

import os
from hydra.experimental import compose, initialize
from hydra.core.global_hydra import GlobalHydra
from omegaconf import OmegaConf
import pytorch_lightning as pl
from nemo.collections.common.callbacks import LogEpochTimeCallback
from nemo.collections.tts.models import TalkNetPitchModel
from nemo.core.config import hydra_runner
from nemo.utils.exp_manager import exp_manager

def train(cfg):
    cfg.sample_rate = 22050
    cfg.train_dataset = "trainfiles.json"
    cfg.validation_datasets = "valfiles.json"
    cfg.durs_file = os.path.join(output_dir, "durations.pt")
    cfg.f0_file = os.path.join(output_dir, "f0s.pt")
    cfg.trainer.accelerator = "dp"
    cfg.trainer.max_epochs = epochs
    cfg.trainer.check_val_every_n_epoch = 5
    cfg.model.f0_mean=f0_mean
    cfg.model.f0_std=f0_std
    cfg.model.train_ds.dataloader_params.batch_size = batch_size
    cfg.model.validation_ds.dataloader_params.batch_size = batch_size
    cfg.model.optim.lr = learning_rate
    cfg.model.optim.sched.min_lr = min_learning_rate
    cfg.exp_manager.exp_dir = output_dir

    # Find checkpoints
    ckpt_path = ""
    if load_checkpoints:
      path0 = os.path.join(output_dir, "TalkNetPitch")
      if os.path.exists(path0):
          path1 = sorted(os.listdir(path0))
          for i in range(len(path1)):
              path2 = os.path.join(path0, path1[-(1+i)], "checkpoints")
              if os.path.exists(path2):
                  match = [x for x in os.listdir(path2) if "last.ckpt" in x]
                  if len(match) > 0:
                      ckpt_path = os.path.join(path2, match[0])
                      print("Resuming training from " + match[0])
                      break
    
    if ckpt_path != "":
        trainer = pl.Trainer(**cfg.trainer, resume_from_checkpoint = ckpt_path)
        model = TalkNetPitchModel(cfg=cfg.model, trainer=trainer)
    else:
        warmstart_path = "/content/talknet_pitch.nemo"
        trainer = pl.Trainer(**cfg.trainer)
        model = TalkNetPitchModel.restore_from(warmstart_path, override_config_path=cfg)
        model.set_trainer(trainer)
        model.setup_training_data(cfg.model.train_ds)
        model.setup_validation_data(cfg.model.validation_ds)
        model.setup_optimization(cfg.model.optim)
        print("Warm-starting from " + warmstart_path)
    exp_manager(trainer, cfg.get('exp_manager', None))
    trainer.callbacks.extend([pl.callbacks.LearningRateMonitor(), LogEpochTimeCallback()])  # noqa
    trainer.fit(model)

GlobalHydra().clear()
initialize(config_path="conf")
cfg = compose(config_name="talknet-pitch")
train(cfg)

[NeMo W 2022-08-14 10:25:50 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:43: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
      deprecation_warning(message=message)
    
[NeMo W 2022-08-14 10:25:50 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:48: UserWarning: 
    The version_base parameter is not specified.
    Please specify a compatability version level, or None.
    Will assume defaults for version 1.1
      caller_stack_depth=caller_stack_depth + 1,
    
[NeMo W 2022-08-14 10:25:50 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/hydra/experimental/compose.py:25: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
      deprecation_warning(message=message)
    
INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: Fal

[NeMo I 2022-08-14 10:25:50 modelPT:439] Model TalkNetPitchModel was successfully restored from /content/talknet_pitch.nemo.
[NeMo I 2022-08-14 10:25:50 collections:173] Dataset loaded with 395 files totalling 0.22 hours
[NeMo I 2022-08-14 10:25:50 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2022-08-14 10:25:51 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2022-08-14 10:25:51 collections:173] Dataset loaded with 45 files totalling 0.02 hours
[NeMo I 2022-08-14 10:25:51 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2022-08-14 10:25:51 modelPT:661] The lightning trainer received accelerator: dp. We recommend to use 'ddp' instead.


...

INFO:pytorch_lightning.utilities.distributed:Epoch 39, global step 519: val_body_mae reached 48.11744 (best 47.08946), saving model to "/content/drive/My Drive/talknet/trained/mizuki/TalkNetPitch/2022-08-14_10-24-59/checkpoints/TalkNetPitch--val_body_mae=48.12-epoch=39.ckpt" as top 3


Validating: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.distributed:Epoch 44, global step 584: val_body_mae was not in top 3


Validating: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.distributed:Epoch 49, global step 649: val_body_mae reached 47.79418 (best 47.08946), saving model to "/content/drive/My Drive/talknet/trained/mizuki/TalkNetPitch/2022-08-14_10-24-59/checkpoints/TalkNetPitch--val_body_mae=47.79-epoch=49.ckpt" as top 3
INFO:pytorch_lightning.utilities.distributed:Saving latest checkpoint...


In [ ]:
#@markdown **Step 9:** Train spectrogram generator. 200+ epochs are recommended. 

#@markdown This is the slowest of the three models to train, and the hardest to
#@markdown get good results from. If your character sounds noisy or robotic,
#@markdown try improving the dataset, or adjusting the epochs and learning rate.

epochs = 200 #@param {type:"integer"}

#@markdown If CUDA runs out of memory, try the following:
#@markdown * Click on Runtime -> Restart runtime, re-run step 3, and try again.
#@markdown * If that doesn't help, reduce the batch size (default 32).
batch_size = 16 #@param {type:"integer"}

#@markdown Advanced settings. You can probably leave these at their defaults (1e-3, 3e-6, empty, checked).
learning_rate = 1e-3 #@param {type:"number"}
min_learning_rate = 3e-6 #@param {type:"number"}
pretrained_path = "" #@param {type:"string"}
load_checkpoints = True #@param {type:"boolean"}

import os
from hydra.experimental import compose, initialize
from hydra.core.global_hydra import GlobalHydra
from omegaconf import OmegaConf
import pytorch_lightning as pl
from nemo.collections.common.callbacks import LogEpochTimeCallback
from nemo.collections.tts.models import TalkNetSpectModel
from nemo.core.config import hydra_runner
from nemo.utils.exp_manager import exp_manager

def train(cfg):
    cfg.sample_rate = 22050
    cfg.train_dataset = "trainfiles.json"
    cfg.validation_datasets = "valfiles.json"
    cfg.durs_file = os.path.join(output_dir, "durations.pt")
    cfg.f0_file = os.path.join(output_dir, "f0s.pt")
    cfg.trainer.accelerator = "dp"
    cfg.trainer.max_epochs = epochs
    cfg.trainer.check_val_every_n_epoch = 5
    cfg.model.train_ds.dataloader_params.batch_size = batch_size
    cfg.model.validation_ds.dataloader_params.batch_size = batch_size
    cfg.model.optim.lr = learning_rate
    cfg.model.optim.sched.min_lr = min_learning_rate
    cfg.exp_manager.exp_dir = output_dir

    # Find checkpoints
    ckpt_path = ""
    if load_checkpoints:
      path0 = os.path.join(output_dir, "TalkNetSpect")
      if os.path.exists(path0):
          path1 = sorted(os.listdir(path0))
          for i in range(len(path1)):
              path2 = os.path.join(path0, path1[-(1+i)], "checkpoints")
              if os.path.exists(path2):
                  match = [x for x in os.listdir(path2) if "last.ckpt" in x]
                  if len(match) > 0:
                      ckpt_path = os.path.join(path2, match[0])
                      print("Resuming training from " + match[0])
                      break
    
    if ckpt_path != "":
        trainer = pl.Trainer(**cfg.trainer, resume_from_checkpoint = ckpt_path)
        model = TalkNetSpectModel(cfg=cfg.model, trainer=trainer)
    else:
        if pretrained_path != "":
            warmstart_path = pretrained_path
        else:
            warmstart_path = "/content/talknet_spect.nemo"
        trainer = pl.Trainer(**cfg.trainer)
        model = TalkNetSpectModel.restore_from(warmstart_path, override_config_path=cfg)
        model.set_trainer(trainer)
        model.setup_training_data(cfg.model.train_ds)
        model.setup_validation_data(cfg.model.validation_ds)
        model.setup_optimization(cfg.model.optim)
        print("Warm-starting from " + warmstart_path)
    exp_manager(trainer, cfg.get('exp_manager', None))
    trainer.callbacks.extend([pl.callbacks.LearningRateMonitor(), LogEpochTimeCallback()])  # noqa
    trainer.fit(model)

GlobalHydra().clear()
initialize(config_path="conf")
cfg = compose(config_name="talknet-spect")
train(cfg)

[NeMo W 2022-08-14 10:28:30 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:43: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
      deprecation_warning(message=message)
    
[NeMo W 2022-08-14 10:28:30 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:48: UserWarning: 
    The version_base parameter is not specified.
    Please specify a compatability version level, or None.
    Will assume defaults for version 1.1
      caller_stack_depth=caller_stack_depth + 1,
    
[NeMo W 2022-08-14 10:28:30 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/hydra/experimental/compose.py:25: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
      deprecation_warning(message=message)
    
INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: Fal

[NeMo I 2022-08-14 10:28:30 features:252] PADDING: 1
[NeMo I 2022-08-14 10:28:30 features:269] STFT using torch
[NeMo I 2022-08-14 10:28:31 modelPT:439] Model TalkNetSpectModel was successfully restored from /content/talknet_spect.nemo.
[NeMo I 2022-08-14 10:28:31 collections:173] Dataset loaded with 395 files totalling 0.22 hours
[NeMo I 2022-08-14 10:28:31 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2022-08-14 10:28:31 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2022-08-14 10:28:31 collections:173] Dataset loaded with 45 files totalling 0.02 hours
[NeMo I 2022-08-14 10:28:31 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2022-08-14 10:28:31 modelPT:661] The lightning trainer received accelerator: dp. We recommend to use 'ddp' instead.


...

INFO:pytorch_lightning.utilities.distributed:Epoch 189, global step 4749: val_loss was not in top 3


Validating: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.distributed:Epoch 194, global step 4874: val_loss was not in top 3


Validating: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.distributed:Epoch 199, global step 4999: val_loss was not in top 3
INFO:pytorch_lightning.utilities.distributed:Saving latest checkpoint...


In [ ]:
#@markdown **Step 10:** Generate GTA spectrograms. This will help HiFi-GAN learn what your TalkNet model sounds like.

#@markdown If this step fails, make sure you've finished training the spectrogram generator.

import sys
import os
import torch
import numpy as np
from tqdm import tqdm
from nemo.collections.tts.models import TalkNetSpectModel
import shutil

def fix_paths(inpath):
    output = ""
    with open(inpath, "r", encoding="utf8") as f:
        for l in f.readlines():
            if l[:5].lower() != "wavs/":
                output += "wavs/" + l
            else:
                output += l
    with open(inpath, "w", encoding="utf8") as w:
        w.write(output)

shutil.copyfile(train_filelist, "/content/hifi-gan/training.txt")
shutil.copyfile(val_filelist, "/content/hifi-gan/validation.txt")
fix_paths("/content/hifi-gan/training.txt")
fix_paths("/content/hifi-gan/validation.txt")
fix_paths("/content/allfiles.txt")

os.chdir('/content')
indir = "wavs"
outdir = "hifi-gan/wavs"
if not os.path.exists(outdir):
    os.mkdir(outdir)

model_path = ""
path0 = os.path.join(output_dir, "TalkNetSpect")
if os.path.exists(path0):
    path1 = sorted(os.listdir(path0))
    for i in range(len(path1)):
        path2 = os.path.join(path0, path1[-(1+i)], "checkpoints")
        if os.path.exists(path2):
            match = [x for x in os.listdir(path2) if "TalkNetSpect.nemo" in x]
            if len(match) > 0:
                model_path = os.path.join(path2, match[0])
                break
assert model_path != "", "TalkNetSpect.nemo not found"

dur_path = os.path.join(output_dir, "durations.pt")
f0_path = os.path.join(output_dir, "f0s.pt")

model = TalkNetSpectModel.restore_from(model_path)
model.eval()
with open("allfiles.txt", "r", encoding="utf-8") as f:
    dataset = f.readlines()
durs = torch.load(dur_path)
f0s = torch.load(f0_path)

for x in tqdm(dataset):
    x_name = os.path.splitext(os.path.basename(x.split("|")[0].strip()))[0]
    x_tokens = model.parse(text=x.split("|")[1].strip())
    x_durs = (
        torch.stack(
            (
                durs[x_name]["blanks"],
                torch.cat((durs[x_name]["tokens"], torch.zeros(1).int())),
            ),
            dim=1,
        )
        .view(-1)[:-1]
        .view(1, -1)
        .to("cuda:0")
    )
    x_f0s = f0s[x_name].view(1, -1).to("cuda:0")
    x_spect = model.force_spectrogram(tokens=x_tokens, durs=x_durs, f0=x_f0s)
    rel_path = os.path.splitext(x.split("|")[0].strip())[0][5:]
    abs_dir = os.path.join(outdir, os.path.dirname(rel_path))
    if abs_dir != "" and not os.path.exists(abs_dir):
        os.makedirs(abs_dir, exist_ok=True)
    np.save(os.path.join(outdir, rel_path + ".npy"), x_spect.detach().cpu().numpy())


[NeMo W 2022-08-14 11:21:09 modelPT:139] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.asr.data.audio_to_text.AudioToCharWithDursF0Dataset
      manifest_filepath: trainfiles.json
      max_duration: null
      min_duration: 0.1
      int_values: false
      load_audio: true
      normalize: false
      sample_rate: 22050
      trim: false
      durs_file: /content/drive/My Drive/talknet/trained/mizuki/durations.pt
      f0_file: /content/drive/My Drive/talknet/trained/mizuki/f0s.pt
      blanking: true
      vocab:
        notation: phonemes
        punct: true
        spaces: true
        stresses: false
        add_blank_at: last
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 16
      num_workers: 4
    
[NeMo W 2022-08-14 11:21:09 modelPT:146] If you intend to

[NeMo I 2022-08-14 11:21:09 features:252] PADDING: 1
[NeMo I 2022-08-14 11:21:09 features:269] STFT using torch
[NeMo I 2022-08-14 11:21:09 modelPT:439] Model TalkNetSpectModel was successfully restored from /content/drive/My Drive/talknet/trained/mizuki/TalkNetSpect/2022-08-14_10-24-59/checkpoints/TalkNetSpect.nemo.


100%|██████████| 440/440 [00:12<00:00, 35.00it/s]


In [ ]:
#@markdown **Step 11:** Train HiFi-GAN. 2,000+ steps are recommended.
#@markdown Stop this cell to finish training the model.

#@markdown If CUDA runs out of memory, click on Runtime -> Restart runtime, re-run step 3, and try again.
#@markdown If this step still fails to start, make sure step 10 finished successfully.

#@markdown Note: If the training process starts at step 2500000, delete the HiFiGAN folder and try again.

import gdown
d = 'https://drive.google.com/uc?id='

os.chdir('/content/hifi-gan')
assert os.path.exists("wavs"), "Spectrogram folder not found"

if not os.path.exists(os.path.join(output_dir, "HiFiGAN")):
    os.makedirs(os.path.join(output_dir, "HiFiGAN"))
if not os.path.exists(os.path.join(output_dir, "HiFiGAN", "do_00000000")):
    print("Downloading universal model...")
    gdown.download(d+"1qpgI41wNXFcH-iKq1Y42JlBC9j0je8PW", os.path.join(output_dir, "HiFiGAN", "g_00000000"), quiet=False)
    gdown.download(d+"1O63eHZR9t1haCdRHQcEgMfMNxiOciSru", os.path.join(output_dir, "HiFiGAN", "do_00000000"), quiet=False)
    start_from_universal = "--warm_start True "
else:
    start_from_universal = ""

!python train.py --fine_tuning True --config config_v1b.json \
{start_from_universal} \
--checkpoint_interval 250 --checkpoint_path "{os.path.join(output_dir, 'HiFiGAN')}" \
--input_training_file "/content/hifi-gan/training.txt" \
--input_validation_file "/content/hifi-gan/validation.txt" \
--input_wavs_dir ".." --input_mels_dir "wavs"


Downloading...
From: https://drive.google.com/uc?id=1qpgI41wNXFcH-iKq1Y42JlBC9j0je8PW
To: /content/drive/My Drive/talknet/trained/mizuki/HiFiGAN/g_00000000
100%|██████████| 55.8M/55.8M [00:00<00:00, 236MB/s]
Downloading...
From: https://drive.google.com/uc?id=1O63eHZR9t1haCdRHQcEgMfMNxiOciSru
To: /content/drive/My Drive/talknet/trained/mizuki/HiFiGAN/do_00000000
100%|██████████| 960M/960M [00:04<00:00, 202MB/s]


Initializing Training Process..
Batch size per GPU : 16
Generator(
  (conv_pre): Conv1d(80, 512, kernel_size=(7,), stride=(1,), padding=(3,))
  (ups): ModuleList(
    (0): ConvTranspose1d(512, 256, kernel_size=(16,), stride=(8,), padding=(4,))
    (1): ConvTranspose1d(256, 128, kernel_size=(16,), stride=(8,), padding=(4,))
    (2): ConvTranspose1d(128, 64, kernel_size=(4,), stride=(2,), padding=(1,))
    (3): ConvTranspose1d(64, 32, kernel_size=(4,), stride=(2,), padding=(1,))
  )
  (resblocks): ModuleList(
    (0): ResBlock1(
      (convs1): ModuleList(
        (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
        (2): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(5,), dilation=(5,))
      )
      (convs2): ModuleList(
        (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,)

In [ ]:
#@markdown **Step 12:** Package the models. They'll be saved to the output directory as [character_name]_TalkNet.zip.

character_name = "mizuki" #@param {type:"string"}

#@markdown When done, generate a Drive share link, with permissions set to "Anyone with the link". 
#@markdown You can then use it with the [Controllable TalkNet notebook](https://colab.research.google.com/drive/1rWsO-WHjAUnYEmWztI86nS0FvB_ao371?usp=sharing) 
#@markdown by selecting "Custom model" as your character.

#@markdown This cell will also move the training checkpoints and logs to the trash.
#@markdown That should free up roughly 2 GB of space on your Drive (remember to empty your trash).
#@markdown If you wish to keep them, uncheck this box.

delete_checkpoints = True #@param {type:"boolean"}

import shutil
from zipfile import ZipFile

def find_talknet(model_dir):
    ckpt_path = ""
    path0 = os.path.join(output_dir, model_dir)
    if os.path.exists(path0):
        path1 = sorted(os.listdir(path0))
        for i in range(len(path1)):
            path2 = os.path.join(path0, path1[-(1+i)], "checkpoints")
            if os.path.exists(path2):
                match = [x for x in os.listdir(path2) if ".nemo" in x]
                if len(match) > 0:
                    ckpt_path = os.path.join(path2, match[0])
                    break
    assert ckpt_path != "", "Couldn't find " + model_dir
    return ckpt_path

durs_path = find_talknet("TalkNetDurs")
pitch_path = find_talknet("TalkNetPitch")
spect_path = find_talknet("TalkNetSpect")
assert os.path.exists(os.path.join(output_dir, "HiFiGAN", "g_00000000")), "Couldn't find HiFi-GAN"

zip = ZipFile(os.path.join(output_dir, character_name + "_TalkNet.zip"), 'w')
zip.write(durs_path, "TalkNetDurs.nemo")
zip.write(pitch_path, "TalkNetPitch.nemo")
zip.write(spect_path, "TalkNetSpect.nemo")
zip.write(os.path.join(output_dir, "HiFiGAN", "g_00000000"), "hifiganmodel")
zip.write(os.path.join(output_dir, "HiFiGAN", "config.json"), "config.json")
zip.write(os.path.join(output_dir, "f0_info.json"), "f0_info.json")
zip.close()
print("Archived model to " + os.path.join(output_dir, character_name + "_TalkNet.zip"))

if delete_checkpoints:
    shutil.rmtree((os.path.join(output_dir, "TalkNetDurs")))
    shutil.rmtree((os.path.join(output_dir, "TalkNetPitch")))
    shutil.rmtree((os.path.join(output_dir, "TalkNetSpect")))
    shutil.rmtree((os.path.join(output_dir, "HiFiGAN")))


Archived model to /content/drive/My Drive/talknet/trained/mizuki/mizuki_TalkNet.zip
